In [1]:
#python_scraper

In [2]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [7]:
def fetch(url):
    f = urlopen(url)
    encoding = f.info().get_content_charset(failobj = "utf-8")
    html = f.read().decode(encoding)
    return html

In [10]:
def scrape(html):
    books = []
    for partial_html in re.findall(r'<td class="left"><a.*?</td>',html, re.DOTALL):
        #도서의 url 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = "http://www.hanbit.co.kr" + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)

In [5]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [8]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [14]:
df = scrape(html)
df.reset_index(drop = True, inplace = True)
df2 = save("books.db",df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,최신 관리회계
1,http://www.hanbit.co.kr/store/books/look.php?p...,리눅스 입문자를 위한 명령어 사전
2,http://www.hanbit.co.kr/store/books/look.php?p...,파타고니아 이야기
3,http://www.hanbit.co.kr/store/books/look.php?p...,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발"
4,http://www.hanbit.co.kr/store/books/look.php?p...,한 권으로 배우는 작고 예쁜 꽃자수
5,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, 처음 만나는 회로이론(2판)"
6,http://www.hanbit.co.kr/store/books/look.php?p...,안전필수 시스템 제어 설계
7,http://www.hanbit.co.kr/store/books/look.php?p...,러닝 리액트(2판)
8,http://www.hanbit.co.kr/store/books/look.php?p...,업무에 바로 쓰는 SQL 튜닝
9,http://www.hanbit.co.kr/store/books/look.php?p...,데이터 스토리


In [15]:
#scrape_by_lxml

In [16]:
!pip install lxml
!pip install cssselect

In [17]:
import lxml.html

In [18]:
# HTML 파일을 읽어 들이고, getroot() 메서드로 HtmlElement 객체 생성
tree = lxml.html.parse("dp.html")
html = tree.getroot()

In [24]:
# cssselect() 메서드로 a 요소의 리스트를 추출 및 반복 수행
for i in html.cssselect("a"):
    print(i.get("href"),":",i.text)

#gnb : None
#top_search : None
#container : None
https://www.hanbit.co.kr/index.html : None
https://www.hanbit.co.kr/media/ : 한빛미디어
https://www.hanbit.co.kr/academy/ : 한빛아카데미
https://www.hanbit.co.kr/biz/ : 한빛비즈
https://www.hanbit.co.kr/life/ : 한빛라이프
https://www.hanbit.co.kr/edu/ : 한빛에듀
https://www.hanbit.co.kr/realtime/ : 리얼타임
https://www.hanbit.co.kr/textbook/ : 한빛정보교과서
https://www.hanbit.co.kr/rent/ : 한빛대관서비스
https://www.hanbit.co.kr/member/login.html : 로그인
https://www.hanbit.co.kr/member/member_agree.html : 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html : 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html : 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e : ENGLISH
https://www.hanbit.co.kr/index.html : 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html : BRAND
https://www.hanbit.co.kr/channel/channel_submain.html : Channel.H
https://www.hanbit.co.kr/store/store_submain.html : STORE
https://www.hanbit.co.kr/support/help_info.html : SUPPORT
https://www.ha

In [ ]:
#scrape_by_bs4

In [25]:
from bs4 import BeautifulSoup

In [31]:
with open("dp.html",encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

In [32]:
for i in soup.find_all("a"):
    print(i.get("href"),i.text)

#gnb 메뉴 바로가기
#top_search 검색 및 카테고리 바로가기
#container 본문 바로가기
https://www.hanbit.co.kr/index.html HOME
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
https://www.hanbit.co.kr/store/store_submain.html STORE
https://www.hanbit.co.kr/support/help_info.html SUPPORT
https://www.hanbit.co.kr/event/current/cur